<a href="https://colab.research.google.com/github/thabresh-s/Data-Science/blob/main/Stock%20Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install pandas
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import yfinance as yf
import pandas as pd
from openpyxl import Workbook


In [ ]:
def get_ohlc_data(ticker, start_date, end_date):
    """
    Function to extract 15-minute OHLC data for a given ticker and date range.
    """
    data = yf.download(ticker, start=start_date, end=end_date, interval='15m')
    return data['Open'], data['High'], data['Low'], data['Close']

def calculate_signal_2candles(df, difference):
    """
    Function to calculate the signal after 2 candles.
    """
    signal = ''
    if (df['Close'][1] > df['High'][0]) and ((df['High'][1] - df['Close'][1])/df['High'][1] < difference):
        signal = 'Bullish'
    elif (df['Close'][1] < df['Low'][0]) and ((df['Close'][1] - df['Low'][1])/df['Low'][1] < difference):
        signal = 'Bearish'
    else:
        signal = 'Neutral'
    return signal

def calculate_signal_3candles(df, difference):
    """
    Function to calculate the signal after 3 candles.
    """
    signal = ''
    if (df['Close'][2] > max(df['High'][0], df['High'][1])) and ((df['High'][2] - df['Close'][2])/df['High'][2] < difference):
        signal = 'Bullish'
    elif (df['Close'][2] < min(df['Low'][0], df['Low'][1])) and ((df['Close'][2] - df['Low'][2])/df['Low'][2] < difference):
        signal = 'Bearish'
    else:
        signal = 'Neutral'
    return signal

def calculate_price_diff(df):
    """
    Function to calculate the price difference between 10AM price and day close price.
    """
    price_diff = df['Close'][df.index[-1]] - df['Close'][df.index[0]]
    return price_diff


In [ ]:
tickers = ['^NSEI', '^NSEBANK'] # Ticker symbols for Nifty futures and BN futures
start_date = '2023-03-01' # Start date of data extraction
end_date = '2023-03-09' # End date of data extraction
difference = 0.0005 # Difference threshold for signal identification

# Extract OHLC data for Nifty futures and BN futures
nifty_open, nifty_high, nifty_low, nifty_close = get_ohlc_data(tickers[0], start_date, end_date)
bn_open, bn_high, bn_low, bn_close = get_ohlc_data(tickers[1], start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# Create a Pandas DataFrame to store the extracted data
df = pd.DataFrame({
    'Date': nifty_close.index,
    'Nifty Open': nifty_open,
    'Nifty High': nifty_high,
    'Nifty Low': nifty_low,
    'Nifty Close': nifty_close,
    'BN Open': bn_open,
    'BN High': bn_high,
    'BN Low': bn_low,
    'BN Close': bn_close
})

# Set the 'Date' column as the index of the DataFrame
df.set_index('Date', inplace=True)

In [ ]:
# Define function to calculate signal after 2 candles
def calculate_signal_2candles(df, difference):
    if df.iloc[1]['Close'] > df.iloc[0]['High'] and abs(df.iloc[1]['Close'] - df.iloc[1]['High'])/df.iloc[1]['Close'] < difference:
        return 'Bullish'
    elif df.iloc[1]['Close'] < df.iloc[0]['Low'] and abs(df.iloc[1]['Close'] - df.iloc[1]['Low'])/df.iloc[1]['Close'] < difference:
        return 'Bearish'
    else:
        return 'N/A'

# Define function to calculate signal after 3 candles
def calculate_signal_3candles(df, difference):
    if df.iloc[2]['Close'] > max(df.iloc[0]['High'], df.iloc[1]['High']) and abs(df.iloc[2]['Close'] - max(df.iloc[0]['High'], df.iloc[1]['High']))/df.iloc[2]['Close'] < difference:
        return 'Bullish'
    elif df.iloc[2]['Close'] < min(df.iloc[0]['Low'], df.iloc[1]['Low']) and abs(df.iloc[2]['Close'] - min(df.iloc[0]['Low'], df.iloc[1]['Low']))/df.iloc[2]['Close'] < difference:
        return 'Bearish'
    else:
        return 'N/A'

In [ ]:
# Apply the signals to identify the market trend for each day
for i in range(len(df)):
    signal_2candles = df.loc[df.index[i], 'Signal after 2 candles']
    signal_3candles = df.loc[df.index[i], 'Signal after 3 candles']
    
    if signal_2candles == 'Bullish' and signal_3candles == 'Bullish':
        df.loc[df.index[i], 'Trend'] = 'Strongly Bullish'
    elif signal_2candles == 'Bearish' and signal_3candles == 'Bearish':
        df.loc[df.index[i], 'Trend'] = 'Strongly Bearish'
    elif signal_2candles == 'Bullish' or signal_3candles == 'Bullish':
        df.loc[df.index[i], 'Trend'] = 'Mildly Bullish'
    elif signal_2candles == 'Bearish' or signal_3candles == 'Bearish':
        df.loc[df.index[i], 'Trend'] = 'Mildly Bearish'
    else:
        df.loc[df.index[i], 'Trend'] = 'Neutral'

KeyError: ignored

In [ ]:
# Define function to get the 10AM price based on the market trend
def get_10am_price(date, trend, nifty_df, bn_df):
    if trend == 'Strongly Bullish':
        return max(nifty_df.loc[date]['Open'], bn_df.loc[date]['Open'])
    elif trend == 'Bullish':
        return (nifty_df.loc[date]['Open'] + bn_df.loc[date]['Open']) / 2
    elif trend == 'Neutral':
        return (nifty_df.loc[date]['Open'] + bn_df.loc[date]['Open'] + nifty_df.loc[date]['Close'] + bn_df.loc[date]['Close']) / 4
    elif trend == 'Bearish':
        return (nifty_df.loc[date]['Close'] + bn_df.loc[date]['Close']) / 2
    elif trend == 'Strongly Bearish':
        return min(nifty_df.loc[date]['Open'], bn_df.loc[date]['Open'])
    else:
        return None

In [ ]:
# Calculate the price difference between 10AM and day close
df['10AM Price'] = df.apply(lambda row: get_10am_price(row.name, row['Trend'], nifty_df, bn_df), axis=1)
df['Price Difference'] = df['Nifty Close'] - df['10AM Price']

In [ ]:
import yfinance as yf
import pandas as pd

# Define function to extract OHLC data for a given ticker
def get_ohlc_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date, progress=False)
    return df['Open'], df['High'], df['Low'], df['Close']

# Define function to calculate signal after 2 candles
def calculate_signal_2candles(df, difference):
    if df.iloc[0]['Open'] < df.iloc[1]['Close'] and df.iloc[1]['Open'] < df.iloc[2]['Close']:
        if abs(df.iloc[2]['Close'] - df.iloc[0]['Open']) > difference:
            return 'Bullish'
    elif df.iloc[0]['Open'] > df.iloc[1]['Close'] and df.iloc[1]['Open'] > df.iloc[2]['Close']:
        if abs(df.iloc[2]['Close'] - df.iloc[0]['Open']) > difference:
            return 'Bearish'
    return 'Neutral'

# Define function to calculate signal after 3 candles
def calculate_signal_3candles(df, difference):
    if df.iloc[0]['Open'] < df.iloc[1]['Close'] and df.iloc[1]['Open'] < df.iloc[2]['Close'] and df.iloc[2]['Open'] < df.iloc[3]['Close']:
        if abs(df.iloc[3]['Close'] - df.iloc[0]['Open']) > difference:
            return 'Bullish'
    elif df.iloc[0]['Open'] > df.iloc[1]['Close'] and df.iloc[1]['Open'] > df.iloc[2]['Close'] and df.iloc[2]['Open'] > df.iloc[3]['Close']:
        if abs(df.iloc[3]['Close'] - df.iloc[0]['Open']) > difference:
            return 'Bearish'
    return 'Neutral'

# Define function to get the 10AM price based on the market trend
def get_10am_price(date, trend, nifty_df, bn_df):
    if trend == 'Strongly Bullish':
        return max(nifty_df.loc[date]['Open'], bn_df.loc[date]['Open'])
    elif trend == 'Bullish':
        return (nifty_df.loc[date]['Open'] + bn_df.loc[date]['Open']) / 2
    elif trend == 'Bearish':
        return (nifty_df.loc[date]['Open'] + bn_df.loc[date]['Open']) / 2
    elif trend == 'Strongly Bearish':
        return min(nifty_df.loc[date]['Open'], bn_df.loc[date]['Open'])

# Define the ticker symbols and date range
tickers = ['^NSEI', '^NSEBANK'] # Ticker symbols for Nifty futures and BN futures
start_date = '2023-03-01' # Start date of data extraction
end_date = '2023-03-09' # End date of data extraction
difference = 0.0005 # Difference threshold for signal identification

# Extract OHLC data for Nifty futures and BN futures
nifty_open, nifty_high, nifty_low, nifty_close = get_ohlc_data(tickers[0], start_date, end_date)
bn_open, bn_high, bn_low, bn_close = get_ohlc_data(tickers[1],start_date, end_date)

In [ ]:
import pandas as pd

start_date = '2023-03-01'
end_date = '2023-03-09'

date_range = pd.date_range(start=start_date, end=end_date)

# Extract OHLC data for Nifty futures and BN futures
nifty_open, nifty_high, nifty_low, nifty_close = get_ohlc_data(tickers[0], start_date, end_date)
bn_open, bn_high, bn_low, bn_close = get_ohlc_data(tickers[1], start_date, end_date)

# Create dataframes for Nifty and BN futures
nifty_df = pd.DataFrame({'Open': nifty_open, 'High': nifty_high, 'Low': nifty_low, 'Close': nifty_close}, index=pd.to_datetime(date_range))
bn_df = pd.DataFrame({'Open': bn_open, 'High': bn_high, 'Low': bn_low, 'Close': bn_close}, index=pd.to_datetime(date_range))

In [ ]:
nifty_df

In [ ]:
import openpyxl

workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Stock Market Analysis"

In [ ]:
worksheet.cell(row=1, column=1, value="Date")
worksheet.cell(row=1, column=2, value="Time")
worksheet.cell(row=1, column=3, value="Nifty Open")
worksheet.cell(row=1, column=4, value="Nifty High")
worksheet.cell(row=1, column=5, value="Nifty Low")
worksheet.cell(row=1, column=6, value="Nifty Close")
worksheet.cell(row=1, column=7, value="BN Open")
worksheet.cell(row=1, column=8, value="BN High")
worksheet.cell(row=1, column=9, value="BN Low")
worksheet.cell(row=1, column=10, value="BN Close")
worksheet.cell(row=1, column=11, value="10AM Price")
worksheet.cell(row=1, column=12, value="9:45AM Signal")
worksheet.cell(row=1, column=13, value="10AM Signal")
worksheet.cell(row=1, column=14, value="Price Difference")

In [ ]:
workbook.save("stock_market_analysis.xlsx")

In [ ]:
df

,Nifty Open,Nifty High,Nifty Low,Nifty Close,BN Open,BN High,BN Low,BN Close
Date,,,,,,,,
2023-03-01 09:15:00,17364.650391,17409.199219,17363.000000,17369.750000,40412.250000,40502.300781,40390.648438,40436.851562
2023-03-01 09:30:00,17369.599609,17421.750000,17362.750000,17405.449219,40442.398438,40531.648438,40412.000000,40487.300781
2023-03-01 09:45:00,17405.750000,17420.650391,17384.599609,17389.400391,40487.050781,40562.648438,40462.699219,40501.300781
2023-03-01 10:00:00,17390.449219,17420.949219,17390.400391,17404.150391,40503.398438,40596.800781,40503.398438,40533.648438
2023-03-01 10:15:00,17404.599609,17420.500000,17389.550781,17418.000000,40531.351562,40604.699219,40499.648438,40598.851562
...,...,...,...,...,...,...,...,...
2023-03-08 14:15:00,17708.300781,17717.199219,17702.800781,17714.250000,41456.898438,41495.601562,41413.550781,41481.199219
2023-03-08 14:30:00,17713.849609,17718.849609,17693.699219,17700.550781,41480.648438,41501.148438,41394.601562,41452.351562
2023-03-08 14:45:00,17700.099609,17710.150391,17693.750000,17707.050781,41451.398438,41459.398438,41422.699219,41447.000000


In [ ]:
import yfinance as yf
import datetime as dt

nifty_ticker = yf.Ticker("^NSEI")
bn_ticker = yf.Ticker("^NSEBANK")

today = dt.date.today()
start_date = today - dt.timedelta(days=5) # get data for last 5 days

In [ ]:
nifty_data = nifty_ticker.history(interval="15m", start=start_date, end=today)
bn_data = bn_ticker.history(interval="15m", start=start_date, end=today)

In [ ]:
def check_signal_945(data):
    if len(data) < 2:
        return None
    second_candle = data.iloc[-2]
    first_candle = data.iloc[-3]
    if second_candle["Close"] > first_candle["High"] and \
            abs((second_candle["High"] - second_candle["Close"]) / second_candle["High"]) < 0.0005:
        return "Bullish"
    elif second_candle["Close"] < first_candle["Low"] and \
            abs((second_candle["Low"] - second_candle["Close"]) / second_candle["Low"]) < 0.0005:
        return "Bearish"
    else:
        return None


def check_signal_10am(data):
    if len(data) < 3:
        return None
    third_candle = data.iloc[-1]
    first_candle = data.iloc[-3]
    second_candle = data.iloc[-2]
    if third_candle["Close"] > max(first_candle["High"], second_candle["High"]) and \
            abs((third_candle["High"] - third_candle["Close"]) / third_candle["High"]) < 0.0005:
        return "Bullish"
    elif third_candle["Close"] < min(first_candle["Low"], second_candle["Low"]) and \
            abs((third_candle["Low"] - third_candle["Close"]) / third_candle["Low"]) < 0.0005:
        return "Bearish"
    else:
        return None

In [ ]:
for result in results:
    date = result[0]
    classification = result[1]
    nifty_price_10am = nifty_data.loc[date].iloc[-1]["Close"]
    nifty_ohlc = nifty_data.loc[date].iloc[0]
    bn_price_10am = bn_data.loc[date].iloc[-1]["Close"]
    bn_ohlc = bn_data.loc[date].iloc[0]
    nifty_price_diff = round(nifty_price_10am - nifty_ohlc["Close"], 2)
    bn_price_diff = round(bn_price_10am - bn_ohlc["Close"], 2)

    print(f"{date}: {classification}, Nifty 10AM Price: {nifty_price_10am}, "
          f"Nifty OHLC: {nifty_ohlc}, Nifty Price Diff: {nifty_price_diff}, "
          f"BN 10AM Price: {bn_price_10am}, BN OHLC: {bn_ohlc}, BN Price Diff: {bn_price_diff}")


In [ ]:
df

,Nifty Open,Nifty High,Nifty Low,Nifty Close,BN Open,BN High,BN Low,BN Close
Date,,,,,,,,
2023-03-01 09:15:00,17364.650391,17409.199219,17363.000000,17369.750000,40412.250000,40502.300781,40390.648438,40436.851562
2023-03-01 09:30:00,17369.599609,17421.750000,17362.750000,17405.449219,40442.398438,40531.648438,40412.000000,40487.300781
2023-03-01 09:45:00,17405.750000,17420.650391,17384.599609,17389.400391,40487.050781,40562.648438,40462.699219,40501.300781
2023-03-01 10:00:00,17390.449219,17420.949219,17390.400391,17404.150391,40503.398438,40596.800781,40503.398438,40533.648438
2023-03-01 10:15:00,17404.599609,17420.500000,17389.550781,17418.000000,40531.351562,40604.699219,40499.648438,40598.851562
...,...,...,...,...,...,...,...,...
2023-03-08 14:15:00,17708.300781,17717.199219,17702.800781,17714.250000,41456.898438,41495.601562,41413.550781,41481.199219
2023-03-08 14:30:00,17713.849609,17718.849609,17693.699219,17700.550781,41480.648438,41501.148438,41394.601562,41452.351562
2023-03-08 14:45:00,17700.099609,17710.150391,17693.750000,17707.050781,41451.398438,41459.398438,41422.699219,41447.000000


In [ ]:
nifty_df

,Open,High,Low,Close
2023-03-01,17360.099609,17467.750000,17345.250000,17450.900391
2023-03-02,17421.500000,17445.800781,17306.000000,17321.900391
2023-03-03,17451.250000,17644.750000,17427.699219,17594.349609
2023-03-04,NaN,NaN,NaN,NaN
2023-03-05,NaN,NaN,NaN,NaN
2023-03-06,17680.349609,17799.949219,17671.949219,17711.449219
2023-03-07,NaN,NaN,NaN,NaN
2023-03-08,17665.750000,17766.500000,17602.250000,17754.400391
2023-03-09,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()

data.nifty_data()

AttributeError: ignored

In [ ]:
import yfinance as yf
import pandas as pd

nifty_fut = yf.Ticker("^NSEI")
nifty_data = nifty_fut.history(interval="15m")
nifty_data.index = pd.to_datetime(nifty_data.index)

bn_fut = yf.Ticker("^NSEBANK")
bn_data = bn_fut.history(interval="15m")
bn_data.index = pd.to_datetime(bn_data.index)

In [ ]:
import pandas as pd

# Create a date range from 1 March 2023 to 10 March 2023
dates = pd.date_range(start="2023-03-01", end="2023-03-10", freq="D")

# Create an empty DataFrame with the required columns
columns = ["Open", "High", "Low", "Close", "Volume"]
nifty_data_new = pd.DataFrame(columns=columns)
bn_data_new = pd.DataFrame(columns=columns)

# Append the new dates to the existing DataFrames
for date in dates:
    nifty_data_new = pd.concat([nifty_data_new, pd.DataFrame(columns=columns, index=[date])])
    bn_data_new = pd.concat([bn_data_new, pd.DataFrame(columns=columns, index=[date])])
    
# Combine the old and new data
nifty_data = pd.concat([nifty_data, nifty_data_new])
bn_data = pd.concat([bn_data, bn_data_new])

In [ ]:
nifty_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2023-02-13 09:15:00+05:30,17859.099609,17879.800781,17829.800781,17834.750000,0,0.0,0.0
2023-02-13 09:30:00+05:30,17834.449219,17834.800781,17801.800781,17830.000000,0,0.0,0.0
2023-02-13 09:45:00+05:30,17830.900391,17830.900391,17772.699219,17773.500000,0,0.0,0.0
2023-02-13 10:00:00+05:30,17772.599609,17782.400391,17736.099609,17746.250000,0,0.0,0.0
2023-02-13 10:15:00+05:30,17746.150391,17761.500000,17738.449219,17757.699219,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-03-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import yfinance as yf
import pandas as pd

start_date = '2023-03-01'
end_date = '2023-03-10'

nifty = yf.download('^NSEI', start=start_date, end=end_date, interval='15m')
bn = yf.download('^NSEBANK', start=start_date, end=end_date, interval='15m')

nifty.to_csv('nifty_15min_data.csv')
bn.to_csv('bn_15min_data.csv')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
nifty=pd.read_csv("/content/nifty_15min_data.csv")

In [ ]:
nifty.to_excel("nifty.xlsx", sheet_name="Testing", index=False)

In [ ]:
bn=pd.read_csv("/content/bn_15min_data.csv")

In [ ]:
bn.to_excel("bn.xlsx", sheet_name="Testing", index=False)

In [ ]:
import yfinance as yf
import pandas as pd

# Download Nifty and BankNifty 15 minute data for 1-Mar-2023 to 10-Mar-2023
nifty_data = yf.download("^NSEI", start="2023-03-01", end="2023-03-10", interval="15m")
bn_data = yf.download("^NSEBANK", start="2023-03-01", end="2023-03-10", interval="15m")

# Combine data into one DataFrame
data = pd.concat([nifty_data["Close"], bn_data["Close"]], axis=1)
data.columns = ["Nifty", "BankNifty"]

# Convert Datetime column to datetime type
data["Datetime"] = pd.to_datetime(data.index)

# Reset index before pivoting
data_reset = data.reset_index()

# Pivot table to have date in rows and time intervals in columns
data_pivot = data_reset.pivot_table(index="Datetime", values=["Nifty", "BankNifty"], aggfunc="last")
data_pivot.columns = [f"{col[1]}_{col[0].strftime('%H:%M')}" for col in data_pivot.columns]

# Save data to Excel
nifty_data.to_excel("nifty_data.xlsx", sheet_name="Testing", index=False)
bn_data.to_excel("bn_data.xlsx", sheet_name="Testing", index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AttributeError: ignored